In [751]:
import msprime
import sys
import numpy as np
import seaborn as sns
from IPython.display import SVG
import matplotlib.pyplot as plt

Coding statistics with tree sequences

In [752]:
#Pairwise diversity with windows
def treediversity(ts,windows=[0,1]):
    stat = [0]*(len(windows)-1)
    n = ts.num_samples
    for tree in ts.trees():
        for site in tree.sites():
            position = site.position
            window_index = sum([windows[i] <= position for i in range(0,len(windows))])
            for mutation in site.mutations:
                #mut = mutation.site(mutation)
                #position = mut.position()
                x = tree.num_samples(mutation.node)
                f = x*(n-x)/(n*(n-1)/2)
                stat[window_index-1] += f/(windows[window_index]-windows[window_index-1])
    return(stat)

windows = [0,0.2,0.33,0.5,0.7,0.9,1]
ts0 = msprime.simulate(5,mutation_rate = 0.1,recombination_rate=0,random_seed=5)

print(treediversity(ts0))
print(ts0.diversity())

[0.6]
0.6


In [756]:
#Tajima's D
def TajimaD(ts):
    n = ts.num_samples
    a1 = sum([1/i for i in range(1,n)])
    b1 = (n+1)/(3*(n-1))
    c1 = b1 - 1/a1
    e1 = c1/a1
    a2 = sum([1/(i**2) for i in range(1,n)])
    b2 = 2*(n**2 + n + 3)/(9*n*(n-1))
    c2 = b2 - (n+2)/(a1*n) + a2/(a1**2)
    e2 = c2/(a1**2 + a2)
    S = ts.segregating_sites()
    theta_pi = treediversity(ts)
    theta_w = S/(a1)
    C = (e1*S + e2*S*(S-1))**(1/2)
    return((theta_pi-theta_w)/C)
pop_configs = [
    msprime.PopulationConfiguration(sample_size=5),
    msprime.PopulationConfiguration(sample_size=5)
]
M= np.array([
    [0,0.1],
    [0.2,0]
])
ts = msprime.simulate(population_configurations=pop_configs,migration_matrix=M,mutation_rate = 0.2,recombination_rate=0.5,random_seed=6)

print(ts.Tajimas_D())
print(TajimaD(ts))

0.12431196051345897
[0.12431196]


In [761]:
#F2

def F2(ts,pops,windows=[0,1]):
    stat = [0]*(len(windows)-1)
    n1 = len(pops[0])
    n2 = len(pops[1])
    for tree in ts.trees(tracked_samples=pops[0]):
        for site in tree.sites():
            position = site.position
            window_index = sum([windows[i] <= position for i in range(0,len(windows))])
            for mutation in site.mutations:
                x1 = 0
                x2 = 0
                for sample in tree.samples(mutation.node):
                    if sample in pops[0]:
                        x1 += 1
                    elif sample in pops[1]:
                        x2 += 1                
                f = (x1*(n2-x2) + x2*(n1-x1))/(n1*n2) - x1*(n1-x1)/(n1*(n1-1)) - x2*(n2-x2)/(n2*(n2-1))
                stat[window_index-1] += f/(windows[window_index]-windows[window_index-1])
    return(stat)
pop_configs = [
    msprime.PopulationConfiguration(sample_size=3),
    msprime.PopulationConfiguration(sample_size=3)
]
M= np.array([
    [0,0.1],
    [0.2,0]
])

ts = msprime.simulate(population_configurations=pop_configs,migration_matrix=M,mutation_rate = 0.1,recombination_rate=0.1,random_seed=9)
    
windows = [0,0.1,0.5,0.66,0.9,1]
pops = [[0,1,2],[3,4,5]]     
print(F2(ts,pops,windows=windows))
print(ts.f2(pops,windows=windows))

[0, -0.2777777777777776, -0.694444444444444, -0.46296296296296274, 0]
[ 0.         -0.27777778 -0.69444444 -0.46296296  0.        ]


In [764]:
#F3
def F3(ts,pops,windows=[0,1]):
    n1 = len(pops[0])
    n2 = len(pops[1])
    n3 = len(pops[2])
    stat = [0]*(len(windows)-1)
    for tree in ts.trees(tracked_samples=pops[0]):
        for site in tree.sites():
            window_index = sum([windows[i] <= site.position for i in range(0,len(windows))])
            for mutation in site.mutations:
                x1 = 0
                x2 = 0
                x3 = 0
                for sample in tree.samples(mutation.node):
                    if sample in pops[0]:
                        x1 += 1
                    elif sample in pops[1]:
                        x2 += 1
                    elif sample in pops[2]:
                        x3 += 1
                f = (1/2)*((x1*(n2-x2) + x2*(n1-x1))/(n1*n2) + (x1*(n3-x3) + x3*(n1-x1))/(n1*n3) - (x2*(n3-x3) + x3*(n2-x2))/(n2*n3) - 2*x1*(n1-x1)/(n1*(n1-1)))
                #f = x1*(x1-1)*(n2-x2)*(n3-x3)/(n1*(n1-1)*n2*n3) - x1*(n1-x1)*(n2-x2)*x3/(n1*(n1-1)*n2*n3)
                stat[window_index - 1] += f/(windows[window_index]-windows[window_index-1])
    return(stat)

pop_configs = [
    msprime.PopulationConfiguration(sample_size=3),
    msprime.PopulationConfiguration(sample_size=3),
    msprime.PopulationConfiguration(sample_size=4)
]
M = np.array([
    [0,0.1,0],
    [0.2,0,0],
    [0.1,0.2,0]
])
ts = msprime.simulate(population_configurations=pop_configs,migration_matrix=M,mutation_rate = 0.7,recombination_rate=0.2,random_seed=1)

pops = [[0,1,2],[3,4,5],[6,7,8,9]] 
windows = [0,0.5,0.6,1]
print(F3(ts,pops,windows=windows))
print(ts.f3(pops,windows=windows))

[-0.4444444444444445, 0.833333333333333, -1.111111111111111]
[-0.44444444  0.83333333 -1.11111111]


In [765]:
#F4
def F4(ts,pops,windows=[0,1]):
    n1 = len(pops[0])
    n2 = len(pops[1])
    n3 = len(pops[2])
    n4 = len(pops[3])
    stat = [0]*(len(windows)-1)
    for tree in ts.trees(tracked_samples=pops[0]):
        for site in tree.sites():
            window_index = sum([windows[i] <= site.position for i in range(0,len(windows))])
            for mutation in site.mutations:
                x1 = 0
                x2 = 0
                x3 = 0
                x4 = 0
                for sample in tree.samples(mutation.node):
                    if sample in pops[0]:
                        x1 += 1
                    elif sample in pops[1]:
                        x2 += 1
                    elif sample in pops[2]:
                        x3 += 1
                    elif sample in pops[3]:
                        x4 += 1
                f = (1/2)*((x1*(n4-x4) + x4*(n1-x1))/(n1*n4) + (x2*(n3-x3) + x3*(n2-x2))/(n2*n3) - (x1*(n3-x3) + x3*(n1-x1))/(n1*n3) - (x2*(n4-x4) + x4*(n2-x2))/(n2*n4))
                stat[window_index - 1] += f/(windows[window_index]-windows[window_index-1])
    return(stat)
pop_configs = [
    msprime.PopulationConfiguration(sample_size=3),
    msprime.PopulationConfiguration(sample_size=3),
    msprime.PopulationConfiguration(sample_size=4),
    msprime.PopulationConfiguration(sample_size=4)
]
M = np.array([
    [0,0.1,0,0.1],
    [0.2,0,0.1,0],
    [0.1,0.2,0,0],
    [0.3,0.1,0,0]
])

ts = msprime.simulate(population_configurations=pop_configs,migration_matrix=M,mutation_rate = 0.6,recombination_rate=0.2,random_seed=3)

pops = [[0,1,2],[3,4,5],[6,7,8,9],[10,11,12,13]] 
print(F4(ts,pops,windows=windows))
print(ts.f4(pops,windows=windows))

[-2.5, -10.000000000000002, 0.625]
[ -2.5   -10.      0.625]


Speed testing for F2

In [795]:
%time
#F2 testing
pop_configs = [
    msprime.PopulationConfiguration(sample_size=100),
    msprime.PopulationConfiguration(sample_size=100)
]
M= np.array([
    [0,0.1],
    [0.2,0]
])
ts_collection = [0]*10000
for i in range(0,10000):
    ts_collection[i] = msprime.simulate(population_configurations=pop_configs,migration_matrix=M,mutation_rate = 0.7,recombination_rate=5,random_seed=i+1)

Wall time: 0 ns


In [796]:
%time
#Inbuilt F2 function
inbuilt_f2_stats = [0]*10000
for i in range(0,10000):
    inbuilt_f2_stats[i] = ts_collection[i].f2([range(0,100),range(100,200)])

Wall time: 0 ns


In [797]:
%time
#My F2 function
my_f2_stats = [0]*10000
for i in range(0,10000):
    my_f2_stats[i] = F2(ts_collection[i],[range(0,100),range(100,200)])

Wall time: 0 ns


Coding F3 with genotype data

In [798]:
def F3gen(matrix,pops):
    num_samples = len(matrix[0])
    num_sites = len(matrix)
    n1 = len(pops[0])
    n2 = len(pops[1])
    n3 = len(pops[2])
    pi_11_count = 0
    pi_12_count = 0
    pi_13_count = 0
    pi_23_count = 0
    for i in range(0,num_sites):
        p1_0count = 0
        p1_1count = 0
        p2_0count = 0
        p2_1count = 0
        p3_0count = 0
        p3_1count = 0
        for j in pops[0]+pops[1]+pops[2]:
            allele = matrix[i][j]
            if j in pops[0]:
                p1_0count += 1 - allele
                p1_1count += allele
            elif j in pops[1]:
                p2_0count += 1 - allele
                p2_1count += allele
            elif j in pops[2]:
                p3_0count += 1 - allele
                p3_1count += allele
        pi_11_count += p1_0count*p1_1count
        pi_12_count += p1_0count*p2_1count + p1_1count*p2_0count
        pi_13_count += p1_0count*p3_1count + p1_1count*p3_0count
        pi_23_count += p2_0count*p3_1count + p2_1count*p3_0count
    pi11 = pi_11_count*2/(n1*(n1-1))
    pi12 = pi_12_count/(n1*n2)
    pi13 = pi_13_count/(n1*n3)
    pi23 = pi_23_count/(n2*n3)
    return((1/2)*(pi12 + pi13 - pi23 - pi11))
pops = [[0,1,2],[3,4,5],[6,7,8,9]] 
windows = [0,0.5,0.6,1]
print(F3(ts,pops))
print(F3gen(ts.genotype_matrix(),pops))

[4.0]
4.0


In [804]:
#F3 testing
pop_configs = [
    msprime.PopulationConfiguration(sample_size=100),
    msprime.PopulationConfiguration(sample_size=100),
    msprime.PopulationConfiguration(sample_size=100)
]
M= np.array([
    [0,0.1,0],
    [0.2,0,0],
    [0.2,0.3,0]
])
pops = [list(range(0,100)),list(range(100,200)),list(range(200,300))]
ts_collection = [0]*1000
for i in range(0,1000):
    ts_collection[i] = msprime.simulate(population_configurations=pop_configs,migration_matrix=M,mutation_rate = 2,recombination_rate=5,random_seed=i+1)

In [805]:
f3_gen_stats = [0]*1000
for i in range(0,1000):
    f3_gen_stats[i] = F3gen(ts_collection[i].genotype_matrix(),pops)